# Python Name‑Matching Exercise

This notebook implements a **name‑matching algorithm** that links candidates from congressional election files to known members of Congress. The task is to match as many people from the `congress_members_with_parties.csv` dataset as possible with records in the `congressional_elections_YYYY.csv` files using name information alone.

Primary Dataset: **congress_members_with_parties.csv**

Election Datasets: **congressional_elections_YYYY.csv** (2019–2025 provided here)

The status column in the election files is **unreliable** and will be ignored. Our goal is to use exact and fuzzy string matching to find the best possible matches.

## Inspect the Data

We begin by loading the primary dataset of Congress members and previewing its structure. Each record includes the member's name, first and last names, title, state, chamber (House or Senate), years of service and party.

We will also load all of the available election files (2019–2025) and inspect their columns. These contain the names of candidates, a URL identifier, their party affiliation, the year of the election and the type of election.

In [ ]:
import pandas as pd
from rapidfuzz import fuzz
import glob
import os
import re

# Helper function to normalise names
def normalize_name(name: str) -> str:
    """Return a normalised version of a name.

    The function lowercases the string, removes punctuation (commas, periods,
    apostrophes, hyphens) and collapses multiple spaces.  The result is a
    canonical form suitable for exact or fuzzy comparison.
    """
    if not isinstance(name, str):
        return ""
    name = name.strip().lower()
    name = re.sub(r"[.,'\-]", " ", name)  # replace punctuation with space
    name = re.sub(r"\s+", " ", name)       # collapse whitespace
    return name


In [ ]:
# Load the primary members dataset
data_dir = '/home/oai/share'
members_path = os.path.join(data_dir, "congress_members_with_parties.csv")
members_df = pd.read_csv(members_path)

# Preview the first few rows
members_df.head()


### Build Member Lookup

To facilitate matching, we compose a **canonical name** for each member in the form `first_name last_name` and normalise it using the helper function defined above.  We build a dictionary that maps each normalised name to the row index of the member in the dataframe.  If duplicates occur, the first occurrence is retained.

In [ ]:
# Compose canonical names and build lookup
tmp_canonical = (members_df["first_name"].fillna("").astype(str).str.strip() + " " +
                 members_df["last_name"].fillna("").astype(str).str.strip()).str.strip()
members_df["canonical_name"] = tmp_canonical

# Build a lookup dictionary from normalised canonical name to member index
member_lookup = {}
for idx, name in enumerate(tmp_canonical):
    norm = normalize_name(name)
    if norm and norm not in member_lookup:
        member_lookup[norm] = idx

print(f"Loaded {len(members_df)} members.")
print(f"Unique normalised names: {len(member_lookup)}")


### Load Election Datasets

Next we discover all election data files in the working directory that match the pattern `congressional_elections_*.csv`.  Each file corresponds to a single election year. We load each into a dataframe and preview a few rows to ensure the columns are consistent across years.

In [ ]:
# Locate and load election datasets
# Collect all election CSV files matching the pattern
raw_files = sorted(glob.glob(os.path.join(data_dir, "congressional_elections_*.csv")))
# Filter out any files with zero observations or unreadable content
election_files = []
for file in raw_files:
    try:
        df = pd.read_csv(file)
        if len(df) > 0:
            election_files.append(file)
        else:
            print(f"Skipping {os.path.basename(file)} because it has no observations.")
    except Exception as e:
        print(f"Skipping {os.path.basename(file)} due to read error: {e}")
print(f"Found {len(election_files)} election files with data: {election_files}")

# Read a sample of each dataset to inspect the structure
for file in election_files:
    df = pd.read_csv(file)
    print(os.path.basename(file), df.shape)
    print(df.head(3), "\n")


## Matching Strategy

The matching logic proceeds in two phases:

1. **Exact Match** – We normalise the candidate's name using the same rules applied to the members. If this normalised string appears in our member lookup dictionary, we record a direct match.
2. **Fuzzy Match** – For candidates that do not have an exact match, we compute a fuzzy similarity score between their normalised name and every member's normalised name using `rapidfuzz.fuzz.token_sort_ratio`. This score ranges from 0 to 100 and is robust to token ordering (e.g., `"Smith, John"` vs. `"John Smith"`). We accept the highest scoring match if it exceeds a chosen threshold (e.g. 85%).

The following cell implements this logic in a reusable function and applies it to each election dataset.

In [ ]:
from typing import List, Tuple

# Precompute a list of normalised member names for fuzzy matching
member_norm_list = [normalize_name(n) for n in members_df["canonical_name"]]

def match_candidates(election_df: pd.DataFrame, threshold: float = 85.0) -> Tuple[pd.DataFrame, List[int]]:
    """Match candidates in an election dataframe to members.

    Parameters
    ----------
    election_df : pd.DataFrame
        The candidate data for a single election year.
    threshold : float, optional
        The minimum fuzzy match score required to accept a match if no exact
        match exists. Defaults to 85.0.

    Returns
    -------
    matched_df : pd.DataFrame
        A dataframe with the original candidate information plus matching
        columns: matched flag, member name, party and confidence score.
    matched_indices : List[int]
        A list of member indices that were matched at least once.
    """

    results = []
    matched_indices: List[int] = []

    for _, row in election_df.iterrows():
        candidate_name = str(row["name"]).strip()
        candidate_party = str(row["party"]).strip()
        candidate_year = int(row["year"]) if "year" in row else None

        # Normalise the candidate name
        c_norm = normalize_name(candidate_name)

        # Default output record
        record = {
            "candidate_name": candidate_name,
            "candidate_party": candidate_party,
            "candidate_year": candidate_year,
            "matched": False,
            "matched_member_name": None,
            "matched_member_party": None,
            "confidence_score": 0.0,
        }

        # Attempt exact match
        if c_norm in member_lookup:
            idx = member_lookup[c_norm]
            mem = members_df.loc[idx]
            record["matched"] = True
            record["matched_member_name"] = mem["name"]
            record["matched_member_party"] = mem["party"]
            record["confidence_score"] = 1.0
            matched_indices.append(idx)
        else:
            # Fuzzy match against all members
            best_score = -1.0
            best_idx = -1
            for i, mem_norm in enumerate(member_norm_list):
                if c_norm and mem_norm:
                    score = fuzz.token_sort_ratio(c_norm, mem_norm)
                    if score > best_score:
                        best_score = score
                        best_idx = i
            if best_score >= threshold:
                mem = members_df.loc[best_idx]
                record["matched"] = True
                record["matched_member_name"] = mem["name"]
                record["matched_member_party"] = mem["party"]
                record["confidence_score"] = best_score / 100.0
                matched_indices.append(best_idx)
            else:
                record["confidence_score"] = best_score / 100.0 if best_score > 0 else 0.0
        results.append(record)

    matched_df = pd.DataFrame(results)
    return matched_df, matched_indices


In [ ]:
all_results = []
all_matched_indices: List[int] = []

for file in election_files:
    election_df = pd.read_csv(file)
    # Ensure there is a "year" column
    if "year" not in election_df.columns:
        try:
            year = int(os.path.splitext(os.path.basename(file)).split("_")[-1].split(".")[0])
        except Exception:
            year = None
        election_df["year"] = year
    matched_df, matched_indices = match_candidates(election_df, threshold=85.0)
    all_results.append(matched_df)
    all_matched_indices.extend(matched_indices)

# Concatenate all results
results_df = pd.concat(all_results, ignore_index=True)

# Preview the first few matches
results_df.head(10)


## Review Unmatched Members

After running the matching process across all available election datasets, we can identify which members from the primary dataset never appeared in any of these elections.  The following code filters out matched indices to produce a list of unmatched members.

In [ ]:
matched_set = set(all_matched_indices)
unmatched_members = members_df.loc[~members_df.index.isin(matched_set)]

print(f"Number of unmatched members: {len(unmatched_members)}")
unmatched_members.head()


### Save Results

Finally, we save the full set of match results to a CSV file along with a separate file listing unmatched members for further analysis.

In [ ]:
# Paths for output files
results_path = os.path.join(data_dir, "name_match_results.csv")
unmatched_path = os.path.join(data_dir, "unmatched_members.csv")

# Save the dataframes
results_df.to_csv(results_path, index=False)
unmatched_members.to_csv(unmatched_path, index=False)

(results_path, unmatched_path)
